# Implementación desde 0 del Método de Máxima Verosimilitud 

Vamos a desarrollar desde 0 todo el código que hay detrás de las funciones que lo calculan automáticamente, para hacer una demostración práctica de lo que hemos visto matemáticamente.

## 1. Definimos la función de entorno L(b)

In [5]:
from IPython.display import display, Math, Latex

In [6]:
display(Math(r'L(\beta)=\sum_{i=1}^n P_i^{y_i}(1-Pi)^{y_i}'))

<IPython.core.display.Math object>

In [8]:
def likelihood(y, pi): #Y es la columna de 0,1
    import numpy as np
    total_sumatorio = 1
    sumatorio_in = list(range(1, len(y)+1))
    for i in range(len(y)):
        sumatorio_in[i] = np.where(y[i]==1, pi[i], 1-pi[i]) #En caso contrario (y=0) nos quedaremos con 1-pi
        total_sumatorio = total_sumatorio * sumatorio_in[i]
    return total_sumatorio

## 2. Calcular las probabilidades Pi para cada observación

In [11]:
print("Pi es la simplificación de P(Xi), que es la Probabilidad para Xi:")
display(Math(r'P_i = P(x_i) = \frac{1}{1+e^{-\sum_{j=0}^k\beta_j\cdot x_{ij}}} '))

Pi es la simplificación de P(Xi), que es la Probabilidad para Xi:


<IPython.core.display.Math object>

In [12]:
#k era el num de columnas
#X mayúscula es el conjunto de puntos (Filas y columnas)
#B vector de todas las betas

def logitprobs(X,beta):
    import numpy as np
    n_rows = np.shape(X)[0]
    n_cols = np.shape(X)[1]
    pi=list(range(1,n_rows+1)) #Creamos un vector con el mismo nº de filas que el dataset
    expon=list(range(1,n_rows+1))
    for i in range(n_rows):
        expon[i] = 0
        for j in range(n_cols):
            ex=X[i][j] * beta[j]
            expon[i] = ex + expon[i]
        with np.errstate(divide="ignore", invalid="ignore"): #En caso de hacer una división imposible, la ignora
            pi[i]=1/(1+np.exp(-expon[i])) #exp es la e
    return pi

## 3. Calcular la matriz diagonal W

In [13]:
display(Math(r'W= diag(P_i \cdot (1-P_i))_{i=1}^n'))

<IPython.core.display.Math object>

In [14]:
def findW(pi):
    import numpy as np
    n = len(pi) #n es la longitud de pi
    W = np.zeros(n*n).reshape(n,n) #Creamos una matriz de 0's con tamaño nxn 
    for i in range(n): #Sustituimos los valores de la diagonal
        print(i)
        W[i,i]=pi[i]*(1-pi[i])
        W[i,i].astype(float) #Lo pasamos a float
    return W

## 4. Obtener la solución de la función logística

Invocamos al método de Newton-Rhapson

In [18]:
display(Math(r"\beta_{n+1} = \beta_n -\frac{f(\beta_n)}{f'(\beta_n)}"))
display(Math(r"f(\beta) = X(Y-P)"))
display(Math(r"f'(\beta) = XWX^T")) #T es la inversa de la matriz X

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [19]:
def logistics(X, Y, limit): #limit = máx de iteraciones que queremos hacer
    import numpy as np
    from numpy import linalg #para hacer la matriz inversa
    nrow = np.shape(X)[0] #Calculamos las filas
    bias = np.ones(nrow).reshape(nrow,1) #Matriz con todo 1's, sólo una columna, n filas.
    X_new = np.append(X, bias, axis = 1) #Añadimos a una matriz la columna de 1's que sirve para cálculos adicionales
    ncol = np.shape(X_new)[1] #Calculamos las columnas
    beta = np.zeros(ncol).reshape(ncol,1) #Creamos una columna de 0's
    root_dif = np.array(range(1,ncol+1)).reshape(ncol,1) #Guardamos las diferencias de las raíces
    iter_i = 10000 #Nº de iteraciones
    while(iter_i>limit): #Mientras que el num de iteración sea superior al límite
        print("Iter:i"+str(iter_i) + ", limit:" + str(limit))
        pi = logitprobs(X_new, beta)
        print("Pi:"+str(pi))
        W = findW(pi)
        print("W:"+str(W)) #Aquí ya tenemos el vector X, la Y, la P y la W, ahora calculamos:
        #Hacemos tantos transpose pq los vectores vienen en filas, los necesitamos en columnas para poder multiplicar
        #Todos tienen que ser el mismo tipo de dato
        num = (np.transpose(np.matrix(X_new))*np.matrix(Y - np.transpose(pi)).transpose()) #Numerador, producto matricial
        den = (np.matrix(np.transpose(X_new))*np.matrix(W)*np.matrix(X_new)) #Denominador
        root_dif = np.array(linalg.inv(den)*num) #Hacemos la inversa del denominador
        beta = beta + root_dif
        print("Beta: "+str(beta))
        iter_i = np.sum(root_dif*root_dif) #Seguimos iterando hasta que haya algo de cambio
        ll = likelihood(Y, pi)
    return beta

## X. Comprobación experimental

Creamos un vector, para saber la forma que tienen los datos

In [20]:
import numpy as np

In [23]:
X = np.array(range(10)).reshape(10,1) #Rango 10, enformato 10 filas, 1 columna
Y = [0,0,0,0,1,0,1,0,1,1] #Creamos a mano el vector de Y's
X

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [24]:
bias = np.ones(10).reshape(10,1) #Creamos otra columan de 1's
X_new = np.append(X,bias,axis=1) #Se la apendizamos a X
X_new

array([[0., 1.],
       [1., 1.],
       [2., 1.],
       [3., 1.],
       [4., 1.],
       [5., 1.],
       [6., 1.],
       [7., 1.],
       [8., 1.],
       [9., 1.]])

In [26]:
a = logistics(X,Y,0.00001) #Llamamos a la función

#Al final está la respuesta

Iter:i10000, limit:1e-05
Pi:[array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5])]
0
1
2
3
4
5
6
7
8
9
W:[[0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25]]
Beta: [[ 0.43636364]
 [-2.36363636]]
Iter:i5.777190082644626, limit:1e-05
Pi:[array([0.08598797]), array([0.12705276]), array([0.18378532]), array([0.2583532]), array([0.35019508]), array([0.45467026]), array([0.56329497]), array([0.66616913]), array([0.75533524]), array([0.826

In [ ]:
Y = 0.66220827 * X -3.69557172